In [1]:
import sys
sys.path.append(sys.path[0][:sys.path[0].rfind('/')])
from StabilizerCircuits import *

# !!!
# IMPORTANT INFO
# GATES AND MEASUREMENT CHANGE THE STATE EVEN IF THERE IS NO '='.

import numpy as np
import copy as copy

# Intro

In [2]:
# initialize state |00..00>
n = 10
table = init_state0(n)
# get stabilizers (first half of the list) and destabilizers (second half) of the state
get_stabilizers(table)

['+ZIIIIIIIII',
 '+IZIIIIIIII',
 '+IIZIIIIIII',
 '+IIIZIIIIII',
 '+IIIIZIIIII',
 '+IIIIIZIIII',
 '+IIIIIIZIII',
 '+IIIIIIIZII',
 '+IIIIIIIIZI',
 '+IIIIIIIIIZ',
 '+XIIIIIIIII',
 '+IXIIIIIIII',
 '+IIXIIIIIII',
 '+IIIXIIIIII',
 '+IIIIXIIIII',
 '+IIIIIXIIII',
 '+IIIIIIXIII',
 '+IIIIIIIXII',
 '+IIIIIIIIXI',
 '+IIIIIIIIIX']

In [3]:
# create circuit of 50 qubis, make some gates and measure!
n = 50 # n > 12
table = init_state0(n)

table = X_gate(table, 9)
table = CNOT(table, 9, 11)
table = Hadamard(table, 2)
table = X_gate(table, 5)
table = iSWAP(table, 5, 6)

for i in range(n):
    table, m = measure(table, i)
    print(i, m)

0 1
1 1
2 1
3 1
4 1
5 1
6 -1
7 1
8 1
9 -1
10 1
11 -1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 1
28 1
29 1
30 1
31 1
32 1
33 1
34 1
35 1
36 1
37 1
38 1
39 1
40 1
41 1
42 1
43 1
44 1
45 1
46 1
47 1
48 1
49 1


# Ideal [5,1,3]

In [10]:
def cycle10(table):
    n = 10
    m_data = np.zeros((n//2))
    m_ancilla = np.zeros((n//2))
    
    even = np.arange(n//2)*2 # ancilla qubits
    odd = np.arange(n//2)*2 + 1 # data qubits
    
    # correction cycle
    for i in even:
        table = Hadamard(table, i)
    for i in even:
        table = iSWAP(table, i, i+1)
    for i in even:
        table = Phase(table, i)
        table = Hadamard(table, i)
    for i in odd:
        table = iSWAP(table, i, (i+1)%n)
    for i in even:
        table = iSWAP(table, i, i+1)
    for i in even:
        table = Z_gate(table, i)
        table = Hadamard(table, i)
    for i in odd:
        table = iSWAP(table, i, (i+1)%n)
    for i in even:
        table = Hadamard(table, i)
        table = Phase(table, i+1)
        
        
    # ancilla measurement
    for i in even:
        table, m_ancilla[i//2] = measure(table, i)
        if m_ancilla[i//2] == -1:
            table = X_gate(table, i)
    
    # data measurement
    table_tmp = copy.copy(table)
    for i in odd:
        table_tmp, m_data[(i-1)//2] = measure(table_tmp, i)
        
    return table, m_data, m_ancilla

In [15]:
N = 10
N_run = 1
N_cycle = 20
M_data = np.zeros((N_run, N_cycle, N//2))
M_ancilla = np.zeros((N_run, N_cycle, N//2))

for n_run in range(0, N_run):
    table = init_state0(N)
    for n_cycle in range(0, N_cycle):
        table, M_data[n_run, n_cycle], M_ancilla[n_run, n_cycle] = cycle10(table)

print(np.prod(M_data, axis = 2))
M_ancilla

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


array([[[-1., -1.,  1., -1., -1.],
        [ 1., -1., -1., -1., -1.],
        [-1., -1., -1.,  1., -1.],
        [-1.,  1., -1., -1., -1.],
        [-1., -1., -1., -1.,  1.],
        [-1., -1.,  1., -1., -1.],
        [ 1., -1., -1., -1., -1.],
        [-1., -1., -1.,  1., -1.],
        [-1.,  1., -1., -1., -1.],
        [-1., -1., -1., -1.,  1.],
        [-1., -1.,  1., -1., -1.],
        [ 1., -1., -1., -1., -1.],
        [-1., -1., -1.,  1., -1.],
        [-1.,  1., -1., -1., -1.],
        [-1., -1., -1., -1.,  1.],
        [-1., -1.,  1., -1., -1.],
        [ 1., -1., -1., -1., -1.],
        [-1., -1., -1.,  1., -1.],
        [-1.,  1., -1., -1., -1.],
        [-1., -1., -1., -1.,  1.]]])